In [9]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import requests,json

In [11]:
trials = "http://ufcstats.com/statistics/fighters?char=a&page=1"
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(trials)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
link = soup.find("td", class_="b-statistics__table-col").a['href']
results = soup.find_all("td", class_="b-statistics__table-col")
link_list = []
for result in results:
    link2 = result.find("a", class_="b-link b-link_style_black")
    if link2 is not None:
        link3 = link2['href']
        link_list.append(link3)
output = []
for x in link_list:
    if x not in output:
        output.append(x)

In [12]:
warrior = list()
with requests.session() as s:
    for url in output:
        react = s.get(url)
        soup = BeautifulSoup(react.content, 'lxml')

        name   = soup.select_one('.b-content__title-highlight').text.strip()
        record = soup.select_one('.b-content__title-record').text.strip().split()[1]
        nick   = soup.select_one('.b-content__Nickname').text.strip()

        fighter = dict(Name=name, Record=record, Nickname=nick)
       
        
        

        for i in soup.select('li .b-list__box-item-title'):
            stat, value = i.text.strip(), i.next.next.strip()
            if stat:
                fighter[stat] = value
            if fighter not in warrior:
                warrior.append(fighter)
            x = json.dumps(fighter, indent=2, sort_keys=True)
        print(warrior)

[{'Name': 'Tom Aaron', 'Record': '5-3-0', 'Nickname': '', 'Height:': '--', 'Weight:': '155 lbs.', 'Reach:': '--', 'STANCE:': '', 'DOB:': 'Jul 13, 1978', 'SLpM:': '0.00', 'Str. Acc.:': '0%', 'SApM:': '0.00', 'Str. Def:': '0%', 'TD Avg.:': '0.00', 'TD Acc.:': '0%', 'TD Def.:': '0%', 'Sub. Avg.:': '0.0'}]
[{'Name': 'Tom Aaron', 'Record': '5-3-0', 'Nickname': '', 'Height:': '--', 'Weight:': '155 lbs.', 'Reach:': '--', 'STANCE:': '', 'DOB:': 'Jul 13, 1978', 'SLpM:': '0.00', 'Str. Acc.:': '0%', 'SApM:': '0.00', 'Str. Def:': '0%', 'TD Avg.:': '0.00', 'TD Acc.:': '0%', 'TD Def.:': '0%', 'Sub. Avg.:': '0.0'}, {'Name': 'Danny Abbadi', 'Record': '4-6-0', 'Nickname': 'The Assassin', 'Height:': '5\' 11"', 'Weight:': '155 lbs.', 'Reach:': '--', 'STANCE:': 'Orthodox', 'DOB:': 'Jul 03, 1983', 'SLpM:': '3.29', 'Str. Acc.:': '38%', 'SApM:': '4.41', 'Str. Def:': '57%', 'TD Avg.:': '0.00', 'TD Acc.:': '0%', 'TD Def.:': '77%', 'Sub. Avg.:': '0.0'}]
[{'Name': 'Tom Aaron', 'Record': '5-3-0', 'Nickname': '', 

In [10]:
warrior

[]

In [16]:
df = pd.DataFrame(warrior)                    

In [18]:
df[['Wins','Losses','Draws']] = df.Record.str.split("-",expand=True,)
df["Wins"] = pd.to_numeric(df["Wins"])
df['Losses'] = pd.to_numeric(df["Losses"])
df["Draws"] = pd.to_numeric(df["Draws"])

In [55]:
print(df.head(1))

        Name Record Nickname Height:   Weight: Reach: STANCE:          DOB:  \
0  Tom Aaron  5-3-0               --  155 lbs.     --          Jul 13, 1978   

  SLpM: Str. Acc.: SApM: Str. Def: TD Avg.: TD Acc.: TD Def.: Sub. Avg.:  \
0  0.00         0%  0.00        0%     0.00       0%       0%        0.0   

   Wins  Losses  Draws   win%  
0     5       3      0  0.625  


In [19]:
df["Str. Acc.:"] = pd.to_numeric(df["Str. Acc.:"].str.replace("%", ""))
df["Str. Def:"] = pd.to_numeric(df["Str. Def:"].str.replace("%", ""))
df["TD Acc.:"] = pd.to_numeric(df["TD Acc.:"].str.replace("%", ""))
df["TD Def.:"] = pd.to_numeric(df["TD Def.:"].str.replace("%", ""))


In [20]:
print(df.head(2))



           Name Record      Nickname Height:   Weight: Reach:   STANCE:  \
0     Tom Aaron  5-3-0                    --  155 lbs.     --             
1  Danny Abbadi  4-6-0  The Assassin  5' 11"  155 lbs.     --  Orthodox   

           DOB: SLpM:  Str. Acc.: SApM:  Str. Def: TD Avg.:  TD Acc.:  \
0  Jul 13, 1978  0.00           0  0.00          0     0.00         0   
1  Jul 03, 1983  3.29          38  4.41         57     0.00         0   

   TD Def.: Sub. Avg.:  Wins  Losses  Draws  
0         0        0.0     5       3      0  
1        77        0.0     4       6      0  
